In [ ]:
%pip install opencv-python
%pip install labelme2coco


In [ ]:
%pip install torch torchvision


In [ ]:
import cv2
import numpy as np
import os

# Read the image and annotation
img_path = r"C:\Users\mason\Endermen\Images\2024-12-29_14.26.33.png"
label_path = r"C:\Users\mason\Endermen\Labelimg\labels\train\2024-12-29_14.26.33.txt"
output_path = r"C:\Users\mason\Endermen\Images\testpoly"

# Create output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# Read image
img = cv2.imread(img_path)
if img is None:
    raise ValueError(f"Could not read image: {img_path}")

height, width = img.shape[:2]

# Read YOLO annotations
with open(label_path, 'r') as f:
    lines = f.readlines()

# Create visualization
viz_img = img.copy()

for line in lines:
    # Parse YOLO format (class x_center y_center width height)
    class_id, x_center, y_center, w, h = map(float, line.strip().split())
    
    # Convert normalized coordinates to pixel values
    x_center = int(x_center * width)
    y_center = int(y_center * height)
    w = int(w * width)
    h = int(h * height)
    
    # Calculate box corners
    x1 = int(x_center - w/2)
    y1 = int(y_center - h/2)
    x2 = int(x_center + w/2)
    y2 = int(y_center + h/2)
    
    # Draw filled polygon with transparency
    overlay = viz_img.copy()
    cv2.rectangle(overlay, (x1, y1), (x2, y2), (0, 255, 0), -1)  # Filled rectangle
    cv2.addWeighted(overlay, 0.3, viz_img, 0.7, 0, viz_img)  # Blend
    
    # Draw box outline
    cv2.rectangle(viz_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # Add label
    label = f"Enderman {class_id}"
    cv2.putText(viz_img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Save visualization
output_file = os.path.join(output_path, "visualized_2024-12-29_14.26.33.png")
cv2.imwrite(output_file, viz_img)

print(f"Saved visualization to: {output_file}")

In [19]:
# Process validation set images
val_file = "C:/Users/mason/Endermen/CVAT/val.txt"
labels_dir = "C:/Users/mason/Endermen/CVAT/labels/val"
output_path = "visualizations"

# Create output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# Read validation image paths
with open(val_file, 'r') as f:
    val_images = f.readlines()

# Process each validation image
for img_path in val_images:
    img_path = img_path.strip()
    
    # Get corresponding label path
    img_name = os.path.basename(img_path)
    label_name = os.path.splitext(img_name)[0] + ".txt"
    label_path = os.path.join(labels_dir, label_name)
    
    # Read image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Could not read image: {img_path}")
        continue

    height, width = img.shape[:2]
    
    # Create visualization
    viz_img = img.copy()
    
    # Read and process labels if they exist
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            lines = f.readlines()
            
        for line in lines:
            # Parse YOLO format (class x_center y_center width height)
            class_id, x_center, y_center, w, h = map(float, line.strip().split())
            
            # Convert normalized coordinates to pixel values
            x_center = int(x_center * width)
            y_center = int(y_center * height)
            w = int(w * width)
            h = int(h * height)
            
            # Calculate box corners
            x1 = int(x_center - w/2)
            y1 = int(y_center - h/2)
            x2 = int(x_center + w/2)
            y2 = int(y_center + h/2)
            
            # Draw filled polygon with transparency
            overlay = viz_img.copy()
            cv2.rectangle(overlay, (x1, y1), (x2, y2), (0, 255, 0), -1)
            cv2.addWeighted(overlay, 0.3, viz_img, 0.7, 0, viz_img)
            
            # Draw box outline
            cv2.rectangle(viz_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Add label
            label = f"Enderman {class_id}"
            cv2.putText(viz_img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Save visualization
    output_file = os.path.join(output_path, f"visualized_{os.path.basename(img_path)}")
    cv2.imwrite(output_file, viz_img)
    print(f"Saved visualization to: {output_file}")


Saved visualization to: visualizations\visualized_2024-12-29_14.31.28.png
Saved visualization to: visualizations\visualized_2024-12-29_14.31.46.png
Saved visualization to: visualizations\visualized_2024-12-29_14.31.58.png
Saved visualization to: visualizations\visualized_2024-12-29_14.32.08.png
Saved visualization to: visualizations\visualized_2024-12-29_14.32.23.png
Saved visualization to: visualizations\visualized_2024-12-29_14.32.37.png


In [21]:
def verify_setup():
    base_dir = Path('CVAT')
    
    # Check data.yaml
    yaml_path = base_dir / 'data.yaml'
    if not yaml_path.exists():
        print(f"Error: data.yaml not found at {yaml_path}")
        return False
        
    # Check directories
    for split in ['train', 'val']:
        img_dir = base_dir / split / 'images'
        label_dir = base_dir / split / 'labels'
        
        if not img_dir.exists():
            print(f"Error: Missing {split}/images directory")
            return False
        if not label_dir.exists():
            print(f"Error: Missing {split}/labels directory")
            return False
            
        # Count files
        images = len(list(img_dir.glob('*.png')) + list(img_dir.glob('*.jpg')))
        labels = len(list(label_dir.glob('*.txt')))
        
        print(f"{split} set:")
        print(f"  Images: {images}")
        print(f"  Labels: {labels}")
    
    return True

# Verify setup before training
if verify_setup():
    print("\nSetup verified, starting training...")
    # Run the training code here
else:
    print("\nPlease fix the setup issues before training")

train set:
  Images: 23
  Labels: 23
val set:
  Images: 6
  Labels: 6

Setup verified, starting training...


In [22]:
from ultralytics import YOLO
import torch
import os
from pathlib import Path

# Check device availability
device = 'cpu'
print(f"Using device: {device}")

# Initialize model
model = YOLO('yolov8n.pt')  # Load pretrained model

# Train the model
try:
    results = model.train(
        data='CVAT/data.yaml',    # Path to data.yaml file
        epochs=100,               # Number of training epochs
        imgsz=640,               # Image size for training
        batch=8,                 # Reduced batch size for CPU
        patience=50,             # Early stopping patience
        device=device,           # Use CPU
        save=True,               # Save training results
        project='runs/train',    # Output directory
        name='yolov8n_endermen', # Run name
        cache=False              # Disable caching for CPU training
    )
    
    print("Training completed successfully!")
    print(f"Results saved in: {model.trainer.save_dir}")

except Exception as e:
    print(f"Error during training: {str(e)}")
    
    # Print additional diagnostic information
    print("\nDiagnostic Information:")
    print(f"Python version: {os.sys.version}")
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    print(f"Current working directory: {os.getcwd()}")
    
    # Verify data.yaml exists
    yaml_path = Path('CVAT/data.yaml')
    if yaml_path.exists():
        print(f"\ndata.yaml found at: {yaml_path.absolute()}")
        with open(yaml_path) as f:
            print("data.yaml contents:")
            print(f.read())
    else:
        print(f"\ndata.yaml not found at: {yaml_path.absolute()}")

Using device: cpu


100%|██████████| 6.25M/6.25M [00:00<00:00, 91.0MB/s]

Ultralytics 8.3.55  Python-3.12.6 torch-2.5.1+cpu CPU (13th Gen Intel Core(TM) i7-13700K)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=CVAT/data.yaml, epochs=100, time=None, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=runs/train, name=yolov8n_endermen, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

 22        [15, 18, 21]  1    751507  ultralytics.nn.modules.head.Detect           [1, [64, 128, 256]]           
Model summary: 225 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'


train: Scanning C:\Users\mason\Endermen\CVAT\train\labels... 23 images, 0 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<00:00, 1755.90it/s]

...

    100/100         0G     0.9107     0.9991      0.843         23        640: 100%|██████████| 3/3 [00:02<00:00,  1.42it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.49it/s]

                   all          6         26      0.684      0.654      0.607      0.315



100 epochs completed in 0.073 hours.
Optimizer stripped from runs\train\yolov8n_endermen\weights\last.pt, 6.2MB
Optimizer stripped from runs\train\yolov8n_endermen\weights\best.pt, 6.2MB

Validating runs\train\yolov8n_endermen\weights\best.pt...
Ultralytics 8.3.55  Python-3.12.6 torch-2.5.1+cpu CPU (13th Gen Intel Core(TM) i7-13700K)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.78it/s]


                   all          6         26      0.684      0.692      0.545      0.348
Speed: 0.2ms preprocess, 14.4ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\train\yolov8n_endermen
Training completed successfully!
Results saved in: runs\train\yolov8n_endermen


In [23]:
# Load the trained model
trained_model = YOLO('runs/train/yolov8n_endermen/weights/best.pt')

# Set path to test images
test_images_path = Path(r'C:\Users\mason\Endermen\cont_end')

# Create output directory for annotated images
output_dir = Path('runs/predict/endermen_detections')
output_dir.mkdir(parents=True, exist_ok=True)

# Run inference on test images
try:
    results = trained_model.predict(
        source=str(test_images_path),
        conf=0.25,  # Confidence threshold
        save=True,  # Save results
        project=str(output_dir.parent),  # Output directory
        name=output_dir.name,  # Run name
        boxes=True,  # Draw boxes
        show_labels=True,  # Show labels
        line_width=2  # Box line width
    )
    
    print(f"Detection completed successfully!")
    print(f"Annotated images saved in: {output_dir}")

except Exception as e:
    print(f"Error during detection: {str(e)}")


WARNING  'boxes' is deprecated and will be removed in in the future. Use 'show_boxes' instead.

image 1/19 C:\Users\mason\Endermen\cont_end\2024-12-29_14.22.35.png: 352x640 4 endermans, 34.2ms
image 2/19 C:\Users\mason\Endermen\cont_end\2024-12-29_14.22.38.png: 352x640 3 endermans, 26.5ms
image 3/19 C:\Users\mason\Endermen\cont_end\2024-12-29_14.22.40.png: 352x640 3 endermans, 32.4ms
image 4/19 C:\Users\mason\Endermen\cont_end\2024-12-29_14.23.35.png: 352x640 3 endermans, 28.7ms
image 5/19 C:\Users\mason\Endermen\cont_end\2024-12-29_14.23.42.png: 352x640 3 endermans, 29.0ms
image 6/19 C:\Users\mason\Endermen\cont_end\2024-12-29_14.24.00.png: 352x640 3 endermans, 28.1ms
image 7/19 C:\Users\mason\Endermen\cont_end\2024-12-29_14.24.11.png: 352x640 4 endermans, 30.0ms
image 8/19 C:\Users\mason\Endermen\cont_end\2024-12-29_14.24.20.png: 352x640 2 endermans, 30.7ms
image 9/19 C:\Users\mason\Endermen\cont_end\2024-12-29_14.24.28.png: 352x640 6 endermans, 26.1ms
image 10/19 C:\Users\mason\Ende